In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import torch
from simclr_model import SimCLR_ResNet18, FullModel
from torch.utils.data import DataLoader
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from finetuning import finetune

torch.manual_seed(0)


cifar100_train = DataLoader(
    datasets.CIFAR100(
        root="./data", transform=transforms.ToTensor(), train=True, download=True
    ),
    batch_size=128,
    shuffle=True,
    num_workers=4,
)
cifar100_test = DataLoader(
    datasets.CIFAR100(
        root="./data", transform=transforms.ToTensor(), train=False, download=True
    ),
    batch_size=128,
    shuffle=False,
    num_workers=4,
)

Files already downloaded and verified
Files already downloaded and verified


# 微调由SimCLR预训练的ResNet18

In [14]:
simclr = SimCLR_ResNet18(128)
simclr.load_state_dict(
    torch.load("./cifar10/checkpoint_90.pth.tar")["model_state_dict"]
)
simclr.backbone.fc = nn.Sequential(
    nn.Linear(512, 512),
    nn.ReLU(),
    nn.Linear(512, 100),
)

In [15]:
finetune(
    model=simclr,
    epoch=20,
    lr=1e-3,
    train_dataloader=cifar100_train,
    test_dataloader=cifar100_test,
    model_dir="./cifar10",
    writer_dir="./cifar_finetuning",
)

  0%|                                                                                                                                                                                                                  | 0/20 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [02:18<00:00,  6.93s/it]


# 微调 ImageNet 预训练的ResNet18

In [5]:
resnet18 = FullModel(num_classes=1000, pretrained=True)
resnet18.backbone.fc = nn.Sequential(
    nn.Linear(512, 512), nn.ReLU(), nn.Linear(512, 100)
)
finetune(
    model=resnet18,
    epoch=20,
    lr=1e-3,
    train_dataloader=cifar100_train,
    test_dataloader=cifar100_test,
    model_dir="./cifar10",
    writer_dir="./resnet18_finetuning",
)

/opt/mamba/envs/nerfstudio/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/mamba/envs/nerfstudio/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
 10%|████████████████████▏                                                                                                                                                                                     | 2/20 [00:19<02:52,  9.58s/it]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [03:04<00:00,  9.24s/it]


# 微调随机初始化的 ResNet18

In [6]:
random_init = FullModel(num_classes=100, pretrained=False)
finetune(
    model=random_init,
    epoch=20,
    lr=1e-3,
    train_dataloader=cifar100_train,
    test_dataloader=cifar100_test,
    model_dir="./random_init",
    writer_dir="./randominit_finetuning",
)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [03:08<00:00,  9.43s/it]


# 微调 STL10 预训练的ResNet18

In [16]:
simclr_stl10 = SimCLR_ResNet18(128)
simclr_stl10.load_state_dict(
    torch.load("./stl10/checkpoint_70.pth.tar")["model_state_dict"]
)
simclr_stl10.backbone.fc = nn.Sequential(
    nn.Linear(512, 512),
    nn.ReLU(),
    nn.Linear(512, 100),
)
finetune(
    model=simclr_stl10,
    epoch=20,
    lr=1e-3,
    train_dataloader=cifar100_train,
    test_dataloader=cifar100_test,
    model_dir="./stl10",
    writer_dir="./stl_finetuning",
)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [02:16<00:00,  6.82s/it]
